# Tensorboard

In [1]:
import torch
from torch.utils.tensorboard import SummaryWriter
from transformers import BertModel, BertTokenizer
import torch.nn as nn

# Create a wrapper class for the BERT model
class BertWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask=None):
        outputs = self.model(input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state

# Load a pre-trained BERT model and tokenizer from Hugging Face
model_name = 'bert-base-uncased'
bert_model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Wrap the BERT model
model = BertWrapper(bert_model)

# Create a SummaryWriter instance
writer = SummaryWriter('runs/huggingface_model_visualization')

# Prepare a dummy input
text = "Hello, this is a sample input for visualization."
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Add the model graph to TensorBoard
writer.add_graph(model, (inputs.input_ids, inputs.attention_mask))

# Close the writer
writer.close()

print(f"Model graph for {model_name} has been added to TensorBoard.")
print("Run 'tensorboard --logdir=runs' to view it.")

In [9]:
help(model.__call__)

Help on method _wrapped_call_impl in module torch.nn.modules.module:

_wrapped_call_impl(*args, **kwargs) method of __main__.BertWrapper instance



# Resnet tensor board

In [2]:
from transformers import AutoImageProcessor, ResNetForImageClassification
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image", trust_remote_code=True)
image = dataset["test"]["image"][0]

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

inputs = processor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])


tiger cat


In [3]:
# inputs

In [5]:
import torch
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn

class ModelWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, pixel_values):
        outputs = self.model(pixel_values)
        return outputs.logits


mw = ModelWrapper(model)

writer = SummaryWriter('.runs/huggingface_model_visualization')

writer.add_graph(mw, inputs['pixel_values'])

writer.close()
print(f"Model graph for ResNet-50 has been added to TensorBoard.")
print("Run 'tensorboard --logdir=.runs' to view it.")

C:\Users\rajab\miniconda3\envs\py3_120\Lib\site-packages\transformers\models\resnet\modeling_resnet.py:91: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:


# stable diffusion

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from torch.utils.tensorboard import SummaryWriter
import torchvision

# Set up the pipeline
model_id = "runwayml/stable-diffusion-v1-5"  # You can change this to your specific model
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# Set up TensorBoard writer
writer = SummaryWriter('runs/stable_diffusion_visualization')

# Your input prompt
prompt = "a photo of an astronaut drinking cola on moon"

# Function to log tensor to TensorBoard
def log_tensor(name, tensor, step):
    if tensor.ndim == 4:  # For image tensors
        writer.add_images(name, tensor, step)
    else:
        writer.add_histogram(name, tensor, step)
    # writer.add_graph(name, tensor, 	)

# Run the pipeline with logging
with torch.no_grad():
    # Tokenize and encode text
    text_inputs = pipe.tokenizer(prompt, padding="max_length", max_length=pipe.tokenizer.model_max_length, return_tensors="pt")
    text_embeddings = pipe.text_encoder(text_inputs.input_ids.to(pipe.device))[0]
    log_tensor("text_embeddings", text_embeddings, 0)

    # Generate random noise
    latents = torch.randn((1, pipe.unet.in_channels, 64, 64)).to(pipe.device)
    log_tensor("initial_latents", latents, 0)

    # Denoise the image
    for i, t in enumerate(pipe.scheduler.timesteps):
        # Expand the latents for classifier-free guidance
        latent_model_input = torch.cat([latents] * 2)
        latent_model_input = pipe.scheduler.scale_model_input(latent_model_input, t)

        # Predict the noise residual
        noise_pred = pipe.unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample
        log_tensor(f"noise_pred_{i}", noise_pred, i)

        # Perform guidance
        noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
        noise_pred = noise_pred_uncond + pipe.guidance_scale * (noise_pred_text - noise_pred_uncond)

        # Compute the previous noisy sample x_t -> x_t-1
        latents = pipe.scheduler.step(noise_pred, t, latents).prev_sample
        log_tensor(f"latents_{i}", latents, i)

    # Decode the image
    image = pipe.vae.decode(latents / pipe.vae.config.scaling_factor, return_dict=False)[0]
    log_tensor("final_image", image, len(pipe.scheduler.timesteps))

    # Save the final image
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
    image = (image * 255).round().astype("uint8")
    torchvision.utils.save_image(torch.from_numpy(image).permute(2, 0, 1) / 255., "generated_image.png")

writer.close()
print("Visualization data has been written to TensorBoard. Run 'tensorboard --logdir=runs' to view it.")

In [ ]:
pipe.__dict__